In [37]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))
from binance import Client
from datetime import datetime 

import src.utils_binance as utils
import src.utils as utils2
import src.myenv as myenv
import time
myenv.telegram_key.append(utils2.get_telegram_key())

import numpy as np
import pandas as pd


In [38]:
utils.get_account_balance()

0.06316951

In [ ]:
#client.get_account_snapshot(type='SPOT')

In [ ]:
symbol = 'AVAXUSDT'
symbol_info = utils.get_client().get_symbol_info(symbol=symbol)
print(f'symbol_info: {symbol_info}')


In [ ]:
price_order = utils.get_client().get_symbol_ticker(symbol=symbol)
symbol_info, symbol_precision, price_precision, step_size, tick_size = utils.get_symbol_info(symbol)
price = round(float(price_order['price']), symbol_precision)

take_profit, stop_loss = utils.calc_take_profit_stop_loss('LONG', price, 1.0, 1)
print((symbol_precision, price_precision, step_size, tick_size))
print((f'price: {price} - take_profit: {take_profit} - stop_loss: {stop_loss}'))

In [ ]:
amount_invested, balance = utils.get_amount_to_invest()  # ok

ledger_params = utils.get_params_operation(np.datetime64('now'), symbol, '1m', 'BUY', 1.0, amount_invested, take_profit, stop_loss,
												price, 30.0, take_profit, 0.0, 1.0, 'LONG', balance, symbol_precision, price_precision, tick_size, step_size)  # ok
print(ledger_params)

In [ ]:
order_buy_id, order_sell_id = utils.register_operation(ledger_params)
print((order_buy_id, order_sell_id))

In [39]:
symbol = 'ATOMUSDT'
interval = '5m'

id_buy = f'{symbol}_{interval}_buy'
id_limit = f'{symbol}_{interval}_limit'
id_stop = f'{symbol}_{interval}_stop'
orders = utils.get_client().get_all_orders(symbol=symbol)

In [40]:
purshased_price = 0.0
stop_loss = 0.0
take_profit = 0.0
df_order = pd.DataFrame(orders)
key = (df_order['clientOrderId'] == id_buy) | (df_order['clientOrderId'] == id_limit) | (df_order['clientOrderId'] == id_stop)
if key.sum() > 0:
	df_order = df_order[key]
	is_purchased = df_order['status'].isin([Client.ORDER_STATUS_NEW, Client.ORDER_STATUS_PARTIALLY_FILLED, Client.ORDER_STATUS_PENDING_CANCEL]).sum() > 0
	if is_purchased:
		has_buy = df_order['clientOrderId'] == id_buy
		if has_buy.sum() > 0:
			purshased_price = float(df_order[has_buy].tail(1)['price'].values[0])
		has_limit = df_order['clientOrderId'] == id_limit
		if has_limit.sum() > 0:
			take_profit = float(df_order[has_limit].tail(1)['price'].values[0])
		has_stop = df_order['clientOrderId'] == id_stop
		if has_stop.sum() > 0:
			stop_loss = float(df_order[has_stop].tail(1)['price'].values[0])

print(f'purshased_price: {purshased_price} - stop_loss: {stop_loss} - take_profit: {take_profit}')
df_order.info()
df_order

purshased_price: 10.8 - stop_loss: 9.029 - take_profit: 11.016
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 20 to 22
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   symbol                   3 non-null      object
 1   orderId                  3 non-null      int64 
 2   orderListId              3 non-null      int64 
 3   clientOrderId            3 non-null      object
 4   price                    3 non-null      object
 5   origQty                  3 non-null      object
 6   executedQty              3 non-null      object
 7   cummulativeQuoteQty      3 non-null      object
 8   status                   3 non-null      object
 9   timeInForce              3 non-null      object
 10  type                     3 non-null      object
 11  side                     3 non-null      object
 12  stopPrice                3 non-null      object
 13  icebergQty               3 non-nul

,symbol,orderId,orderListId,clientOrderId,price,origQty,executedQty,cummulativeQuoteQty,status,timeInForce,type,side,stopPrice,icebergQty,time,updateTime,isWorking,workingTime,origQuoteOrderQty,selfTradePreventionMode
20,ATOMUSDT,2572241869,-1,ATOMUSDT_5m_buy,10.80000000,0.93000000,0.93000000,10.04400000,FILLED,GTC,LIMIT,BUY,0.00000000,0.00000000,1702983042484,1702983045138,True,1702983042484,0.00000000,EXPIRE_MAKER
21,ATOMUSDT,2572242025,95719071,ATOMUSDT_5m_stop,9.02900000,0.93000000,0.00000000,0.00000000,NEW,GTC,STOP_LOSS_LIMIT,SELL,9.50400000,0.00000000,1702983048373,1702983048373,False,-1,0.00000000,EXPIRE_MAKER
22,ATOMUSDT,2572242026,95719071,ATOMUSDT_5m_limit,11.01600000,0.93000000,0.00000000,0.00000000,NEW,GTC,LIMIT_MAKER,SELL,0.00000000,0.00000000,1702983048373,1702983048373,True,1702983048373,0.00000000,EXPIRE_MAKER
